In [ ]:
import tcxparser
import pandas as pd
from datetime import datetime
from os import listdir
from os.path import join

def get_hr_data(tcx_path):
    tcx = tcxparser.TCXParser(tcx_path)
    hr_values = tcx.hr_values()
    time_values = [datetime.strptime(t, '%Y-%m-%dT%H:%M:%S.000Z') for t in tcx.time_values()]
    elapsed = [(tv - time_values[0]).seconds for tv in time_values]
    return pd.DataFrame(data={'elapsed': elapsed, 'hr': hr_values})

def load_data(dir_path):
    tcx_filenames = [f for f in listdir(dir_path) if f.endswith('.tcx')]
    data = [get_hr_data(join(dir_path, f)) for f in tcx_filenames]
    dates = [datetime.strptime(f.split('_')[1], '%Y%m%d') for f in tcx_filenames]
    return data, dates

In [ ]:
activities, activity_dates = load_data('data')

In [ ]:
first_activity = activities[0]
first_activity.head()

In [ ]:
first_activity.tail()

In [ ]:
import bokeh
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.palettes import Blues9, Spectral9, RdYlGn5
from bokeh.models import BoxAnnotation
from bokeh.models import Range1d

In [ ]:
output_notebook()

In [ ]:
TOOLS = 'box_zoom,resize,reset,hover'
    
WALKING = Spectral9[0]
JOGGING = Spectral9[1]
RUNNING = Spectral9[6]

def plot_activity(activity):
    p = figure(plot_width=800, tools=TOOLS)
    p.line(activity.elapsed, activity.hr, line_width=2)
    return p

def plot_activities(activities):
    p = figure(plot_width=800, tools=TOOLS)
    for activity in activities:
        p.line(activity.elapsed, activity.hr, line_width=2)
    return p

def plot_activities_colourful(activities):
    p = figure(plot_width=800, tools=TOOLS)
    colours = list(Blues9); colours.reverse()
    for activity in activities:
        line_colour = colours.pop()
        p.line(activity.elapsed, activity.hr, line_width=2, line_color=line_colour)
    return p

def add_intensity_bands(p):
    opacity = 0.1
    
    p.add_layout(BoxAnnotation(left=0, right=300, fill_alpha=opacity, line_color=WALKING, fill_color=WALKING))
    p.add_layout(BoxAnnotation(left=300, right=600, fill_alpha=opacity, line_color=JOGGING, fill_color=JOGGING))
    p.add_layout(BoxAnnotation(left=600, right=3000, fill_alpha=opacity, line_color=RUNNING, fill_color=RUNNING))
    p.add_layout(BoxAnnotation(left=3000, right=3300, fill_alpha=opacity, line_color=JOGGING, fill_color=JOGGING))
    p.add_layout(BoxAnnotation(left=3300, right=3600, fill_alpha=opacity, line_color=WALKING, fill_color=WALKING))

def focus(p, start, end, hr_min, hr_max):
    p.x_range = Range1d(start, end)
    p.y_range = Range1d(hr_min, hr_max)

In [ ]:
p = plot_activity(first_activity)
show(p)

In [ ]:
p = plot_activity(first_activity)
add_intensity_bands(p)
show(p)

In [ ]:
p = plot_activities(activities)
show(p)

In [ ]:
p = plot_activities_colourful(activities)
show(p)

In [ ]:
p = plot_activities_colourful(activities)
add_intensity_bands(p)
focus(p, 0, 3600, 70, 170)
show(p)

In [ ]:
p = plot_activities_colourful(activities)
add_intensity_bands(p)
focus(p, 240, 600, 80, 140)
show(p)

In [ ]:
p = plot_activities_colourful(activities)
add_intensity_bands(p)
focus(p, 540, 900, 115, 155)
show(p)

In [ ]:
p = plot_activities_colourful(activities)
add_intensity_bands(p)
focus(p, 2940, 3300, 135, 165)
show(p)

In [ ]:
p = plot_activities_colourful(activities)
add_intensity_bands(p)
focus(p, 3240, 3600, 110, 160)
show(p)

In [ ]:
first_activity.elapsed.between(120, 180)

In [ ]:
first_activity[first_activity.elapsed.between(120, 180)]

In [ ]:
def mean_hr_between(activity, start, end):
    data = activity[activity.elapsed.between(start, end)]
    return data.hr.mean()

mean_hr_between(first_activity, 120, 180)

In [ ]:
running_mean = [mean_hr_between(a, 600, 3000) for a in activities]

p = figure(plot_width=800, plot_height=400, x_axis_type="datetime", tools=TOOLS)
p.circle(activity_dates, running_mean, size=10, color=Spectral9[6])
show(p)

In [ ]:
running_steady = [mean_hr_between(a, 2880, 3000) for a in activities]
cooldown_steady = [mean_hr_between(a, 3180, 3300) for a in activities]

p = figure(plot_width=800, plot_height=400, x_axis_type="datetime", tools=TOOLS)
p.circle(activity_dates, running_steady, size=10, color=Spectral9[6])
p.circle(activity_dates, cooldown_steady, size=10, color=Spectral9[1])
show(p)

In [ ]:
walking_steady = [mean_hr_between(a, 3480, 3600) for a in activities]

p = figure(plot_width=800, plot_height=400, x_axis_type="datetime", tools=TOOLS)
p.circle(activity_dates, cooldown_steady, size=10, color=Spectral9[1])
p.circle(activity_dates, walking_steady, size=10, color=Spectral9[0])
show(p)

In [ ]:
p = plot_activity(first_activity)
focus(p, 2940, 3300, 150, 165)
show(p)

In [ ]:
p = plot_activity(first_activity)

running_steady = mean_hr_between(first_activity, 2940, 3000)
p.line(x=[0,3600], y=[running_steady, running_steady], line_color=RUNNING, line_width=2)

jogging_steady = mean_hr_between(first_activity, 3240, 3300)
p.line(x=[0,3600], y=[jogging_steady, jogging_steady], line_color=JOGGING, line_width=2)

focus(p, 2940, 3300, 150, 165)
p.ygrid.grid_line_color = None
p.xgrid.grid_line_alpha = 0.75
p.xgrid.grid_line_dash = [6, 4]
show(p)

In [ ]:
threshold = (running_steady + jogging_steady) / 2
threshold

In [ ]:
p = plot_activity(first_activity)

running_steady = mean_hr_between(first_activity, 2940, 3000)
p.line(x=[0,3600], y=[running_steady, running_steady], line_color=RUNNING, line_width=2)

p.line(x=[0,3600], y=[threshold, threshold], line_color='black', line_width=2, line_dash=[8, 3])

jogging_steady = mean_hr_between(first_activity, 3240, 3300)
p.line(x=[0,3600], y=[jogging_steady, jogging_steady], line_color=JOGGING, line_width=2)

focus(p, 2940, 3300, 150, 165)
p.ygrid.grid_line_color = None
p.xgrid.grid_line_alpha = 0.75
p.xgrid.grid_line_dash = [6, 4]
show(p)

In [ ]:
jogging = first_activity[first_activity.elapsed.between(3000, 3300)]
jogging[jogging.hr < threshold].head()

In [ ]:
jogging[jogging.hr < threshold].iloc[0].elapsed

In [ ]:
threshold_hit_after = jogging[jogging.hr < threshold].iloc[0].elapsed - 3000
threshold_hit_after

In [ ]:
p = plot_activity(first_activity)

running_steady = mean_hr_between(first_activity, 2940, 3000)
p.line(x=[0,3600], y=[running_steady, running_steady], line_color=RUNNING, line_width=1)

p.line(x=[0,3600], y=[threshold, threshold], line_color='black', line_width=2, line_dash=[8, 3])

jogging_steady = mean_hr_between(first_activity, 3240, 3300)
p.line(x=[0,3600], y=[jogging_steady, jogging_steady], line_color=JOGGING, line_width=1)

hit_at = 3000 + threshold_hit_after
p.line(x=[3000,3000], y=[0, 180], line_color=RUNNING, line_width=1)
p.line(x=[hit_at, hit_at], y=[0, 180], line_color='black', line_width=2, line_dash=[8,4])

focus(p, 2940, 3300, 150, 165)
p.ygrid.grid_line_color = None
p.xgrid.grid_line_color = None
show(p)

In [ ]:
def calc_threshold(start_value, end_value, percent):
    delta = end_value - start_value
    return start_value + percent * delta

calc_threshold(running_steady, jogging_steady, 0.5)

In [ ]:
def calc_threshold_offset(activity, start_time, end_time):
    initial_hr = mean_hr_between(activity, start_time - 60, start_time)
    final_hr = mean_hr_between(activity, end_time - 60, end_time)

    threshold = calc_threshold(initial_hr, final_hr, 0.5)
    
    transition = activity[activity.elapsed.between(start_time, end_time)]
    first_sample = transition[transition.hr < threshold].iloc[0]
    
    return first_sample.elapsed - start_time

calc_threshold_offset(first_activity, 3000, 3300)

In [ ]:
run_jog_offsets = [calc_threshold_offset(a, 3000, 3300) for a in activities]
run_jog_offsets

In [ ]:
p = figure(plot_width=800, plot_height=400, x_axis_type="datetime", tools=TOOLS)
p.circle(activity_dates, run_jog_offsets, size=10)
show(p)

In [ ]:
def calc_threshold_offset(activity, start_time, end_time, percent):
    initial_hr = mean_hr_between(activity, start_time - 60, start_time)
    final_hr = mean_hr_between(activity, end_time - 60, end_time)

    threshold = calc_threshold(initial_hr, final_hr, percent)
    
    transition = activity[activity.elapsed.between(start_time, end_time)]
    first_sample = transition[transition.hr < threshold].iloc[0]
    
#     print(first_sample)
    
    return first_sample.elapsed - start_time

calc_threshold_offset(first_activity, 3000, 3300, 0.5)

In [ ]:
[[calc_threshold_offset(a, 3000, 3300, p) for p in [0.1, 0.25, 0.5, 0.75, 0.9]] for a in activities]

In [ ]:
PERCENTAGES = [0.1, 0.25, 0.5, 0.75, 0.9]
p = figure(plot_width=800, plot_height=400, x_axis_type="datetime", tools=TOOLS)

colours = list(RdYlGn5)

for pct in PERCENTAGES:
    colour = colours.pop()
    size = sizes.pop()
    offsets = [calc_threshold_offset(a, 3000, 3300, pct) for a in activities]
    p.circle(activity_dates, offsets, size=12, color=colour)
show(p)